# Julia + DRUM

## Set Up

See [this](https://github.com/datarobot/datarobot-user-models/tree/master/model_templates/inference/julia#setup) for details on setting up Julia.  

## Goal

We'll be using MLJ Pipelines and MLJDecisionTreeInterface to generate an artifact that we can take into DataRobot for the purpose of interence.  It will be necessary to couple this artifact with a `custom.jl` due to usage of MLJ pipelines

## Libraries

The main libraries we'll be using are 
* MLJ - Machine Learning Framework for Julia
* DataFrames
* CSV

When using Julia libraries, you have the option of either `using` or `import`.  They have very different effects, so give [this thread](https://stackoverflow.com/questions/27086159/what-is-the-difference-between-using-and-import-in-julia-when-building-a-mod) a read if you are interested

The main thing to know is that `using` exposes everything within the module's export statement, while `import` does not.  

In [8]:
using DataFrames
using MLJ
using CSV

In [10]:
path_to_data = "data/boston_housing.csv"
df = DataFrame(CSV.File(path_to_data))

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [11]:
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,CRIM,3.61352,0.00632,0.25651,88.9762,0,Float64
2,ZN,11.3636,0.0,0.0,100.0,0,Float64
3,INDUS,11.1368,0.46,9.69,27.74,0,Float64
4,CHAS,0.06917,0,0.0,1,0,Int64
5,NOX,0.554695,0.385,0.538,0.871,0,Float64
6,RM,6.28463,3.561,6.2085,8.78,0,Float64
7,AGE,68.5749,2.9,77.5,100.0,0,Float64
8,DIS,3.79504,1.1296,3.20745,12.1265,0,Float64
9,RAD,9.54941,1,5.0,24,0,Int64


In [12]:
y, X = unpack(df, ==(:MEDV), colname -> true);

## MLJ Pipelines

Pipelines are weird in MLJ (in my opinion).  Each pipeline that is created is basically a type, this means that if you serialize a pipeline, and attempt to reinstantiate, the type needs to be available.  We'll recover this during the review of the `custom.jl` file. 

In [5]:
## features used in model
fl = [:CRIM,:ZN,:INDUS,:CHAS,:NOX,:RM,:AGE,:DIS,:RAD,:TAX,:PTRATIO,:B,:LSTAT]
## feature selector is used
feature_selector = FeatureSelector(features = fl)
## arbitrary imputer
arb_imp = X -> coalesce.(X, -99999)
## categorical encoder
cont_encoder = ContinuousEncoder()
## standardize the features
# ss = Standardizer(features = fl)
DecisionTreeRegressor = @load DecisionTreeRegressor pkg="DecisionTree"
tree = DecisionTreeRegressor() 
pipe = @pipeline feature_selector arb_imp cont_encoder tree name=mypipe;

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/timothy.whittaker/.julia/packages/MLJModels/9mQfs/src/loading.jl:168


import MLJDecisionTreeInterface ✔


## Train and Save the Model

In [13]:
ml = machine(pipe, X,y)
train, test = partition(eachindex(y), 0.7, shuffle=true); # 70:30 split
fit!(ml, rows=train, verbosity=0)

In [ ]:
MLJ.save("tree_pipeline.jlso", ml)

## Custom.jl

The `custom.jl` is necessary for the artifact due to usage of MLJ Pipelines.  The Pipeline we generated is a type and if we attempt to instantiate the pipeline and the type is not available, we will get an error.  This means, that we have to create the same pipeline in our `custom.jl` and expose it at the top level so that when we instantiate the model, it does so properly.  

The content of the `custom.jl` is as follows

```:julia
module Custom

using MLJ, DataFrames
using Base.Filesystem
 
export load_model, score, mypipe
DecisionTreeRegressor = @load DecisionTreeRegressor pkg="DecisionTree"
arb_imp = X -> coalesce.(X, -99999)
@pipeline FeatureSelector arb_imp ContinuousEncoder DecisionTreeRegressor name=mypipe

function load_model(code_dir)
    artifact_path = Filesystem.joinpath(code_dir, "tree_pipeline.jlso")
    model = machine(artifact_path)
    return model
end

function score(data, model; kwargs)
    predictions = predict(model, data)
    return DataFrame(Predictions = predictions)
end

end
```

## Scoring

In [23]:
run(`drum score --code-dir . --target-type regression --input $path_to_data`)

import MLJDecisionTreeInterface ✔
     Predictions
0      24.160000
1      21.483333
2      35.285714
3      32.728571
4      32.728571
..           ...
501    23.860000
502    20.700000
503    25.080000
504    25.080000
505    20.700000

[506 rows x 1 columns]


 Activating environment at `~/Desktop/mlops-experiments/python-julia-sys-image/Project.toml`
[ Info: creating pipeline type
[ Info: For silent loading, specify `verbosity=0`. 
[ Info: successfully included /Users/timothy.whittaker/Desktop/git/datarobot-user-models/model_templates/inference/julia/model1/custom.jl
[ Info: load_model is defined
[ Info: score is defined
[ Info: Main.MLJLibs
[ Info: Loading model via hook
[ Info: loading tree pipeline


Process(`drum score --code-dir ./model1 --target-type regression --input /Users/timothy.whittaker/Desktop/git/datarobot-user-models/tests/testdata/boston_housing.csv`, ProcessExited(0))

## Performance Testing

In [19]:
run(`drum perf-test --code-dir . --target-type regression --input $path_to_data`)

DRUM performance test
Model:      /Users/timothy.whittaker/Desktop/git/datarobot-user-models/model_templates/inference/julia/model1
Data:       /Users/timothy.whittaker/Desktop/git/datarobot-user-models/tests/testdata/boston_housing.csv
# Features: 14
Preparing test data...



Running test case with timeout: 600
Running test case: 68 bytes - 1 samples, 100 iterations
Running test case with timeout: 600
Running test case: 0.1MB - 1529 samples, 50 iterations
Running test case with timeout: 600
Running test case: 10MB - 152926 samples, 5 iterations
Running test case with timeout: 600
Running test case: 50MB - 764634 samples, 1 iterations
Test is done stopping drum server

  size     samples   iters    min     avg     max     used (MB)   total physical
                                                                       (MB)     
68 bytes         1     100   0.009   0.147   13.783     596.797        16384.000
0.1MB         1529      50   0.014   0.016    0.034     625.074        16384.00

tput: terminal attributes: Device not configured



Process(`drum perf-test --code-dir ./model1 --target-type regression --input /Users/timothy.whittaker/Desktop/git/datarobot-user-models/tests/testdata/boston_housing.csv`, ProcessExited(0))

## Validation

In [22]:
run(`drum validation --code-dir . --target-type regression --input $path_to_data`)

import MLJDecisionTreeInterface ✔
import MLJDecisionTreeInterface ✔


 Activating environment at `~/Desktop/mlops-experiments/python-julia-sys-image/Project.toml`
[ Info: creating pipeline type
[ Info: For silent loading, specify `verbosity=0`. 
[ Info: successfully included /Users/timothy.whittaker/Desktop/git/datarobot-user-models/model_templates/inference/julia/model1/custom.jl
[ Info: load_model is defined
[ Info: score is defined
[ Info: transform is defined
[ Info: Main.MLJLibs
[ Info: Loading model via hook
[ Info: loading tree pipeline
 Activating environment at `~/Desktop/mlops-experiments/python-julia-sys-image/Project.toml`
[ Info: creating pipeline type
[ Info: For silent loading, specify `verbosity=0`. 
[ Info: successfully included /Users/timothy.whittaker/Desktop/git/datarobot-user-models/model_templates/inference/julia/model1/custom.jl
[ Info: load_model is defined
[ Info: score is defined
[ Info: transform is defined
[ Info: Main.MLJLibs
[ Info: Loading model via hook
[ Info: loading tree pipeline
┌ Warning: thread = 1 warning: only foun

import MLJDecisionTreeInterface ✔


┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 69. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 70. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 71. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 72. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 73. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 74. Filling remaining columns with `missing`
└ @ CSV ~/.julia/pac

┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 25. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 26. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 27. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 28. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 29. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 30. Filling remaining columns with `missing`
└ @ CSV ~/.julia/pac

...

import MLJDecisionTreeInterface ✔


┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 62. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 63. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 64. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 65. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 66. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 67. Filling remaining columns with `missing`
└ @ CSV ~/.julia/pac

┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 17. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 18. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 19. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 20. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 21. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/CJfFO/src/file.jl:605
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 22. Filling remaining columns with `missing`
└ @ CSV ~/.julia/pac



Validation checks results
      Test case          Status   Details
Basic batch prediction   PASSED          
Null value imputation    PASSED          


Process(`drum validation --code-dir ./model1 --target-type regression --input /Users/timothy.whittaker/Desktop/git/datarobot-user-models/tests/testdata/boston_housing.csv`, ProcessExited(0))